In [18]:
!pip install transformers
!pip install einops
!pip install datasets


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install rouge_score
!pip install evaluate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer,AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
#model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [54]:
import pandas as pd

df=pd.read_csv('faq_updated.csv')
df

,Question,Answer
0,What is a hurricane?,A hurricane is a powerful tropical storm with ...
1,How are hurricanes formed?,Hurricanes form over warm ocean waters when at...
2,What are the stages of hurricane development?,"Tropical depression, tropical storm, hurricane."
3,What is the Saffir-Simpson Hurricane Wind Scale?,It rates hurricanes from Category 1 (weakest) ...
4,How fast do hurricane winds need to be for it ...,Winds must reach at least 74 miles per hour (1...
...,...,...
481,Myth busted! – “But I don't have time to prepa...,Procrastinating flood preparedness can put you...
482,Myth busted! – “My pets will be fine on their ...,Pets are vulnerable during floods. It's essent...
483,Myth busted! – “But I don't want to evacuate a...,While it's natural to want to protect your hom...
484,Myth busted! – “I'll be able to call for help ...,"During a flood, communication infrastructure c..."


In [55]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
df_test.shape

(49, 2)

In [56]:
df_test

,Question,Answer
420,What should breastfeeding mothers do if they e...,Breastfeeding mothers should seek support from...
262,Why is it essential to know your evacuation po...,Knowing your evacuation point and preferred ro...
431,Can you replace lost medicine without a prescr...,If you've lost your medicine but still have yo...
448,What steps should you take if you are outdoors...,If you are outdoors during a severe storm or c...
364,What information should accompany your pet if ...,If you house your pet in a temporary foster ho...
465,Myth busted! – “We’re protected by levees wher...,Levees can provide limited protection to build...
193,What can I do to access emergency information ...,Keep a battery-powered radio with extra batter...
225,Why is it a good idea to create wallet emergen...,Creating wallet emergency cards for household ...
104,What are the key items to have for illuminatio...,Key items for illumination in your emergency k...
229,What community emergency services organization...,Consider joining a community emergency service...


In [57]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [58]:
train_dataset

Dataset({
    features: ['Question', 'Answer', '__index_level_0__'],
    num_rows: 437
})

In [59]:
from datasets import concatenate_datasets


tokenized_inputs = concatenate_datasets([train_dataset, test_dataset]).map(lambda x: tokenizer(x["Question"], truncation=True), batched=True, remove_columns=['Answer', '__index_level_0__'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

Map:   0%|          | 0/486 [00:00<?, ? examples/s]

Max source length: 37


In [60]:
tokenized_targets = concatenate_datasets([train_dataset, test_dataset]).map(lambda x: tokenizer(x["Answer"], truncation=True), batched=True, remove_columns=["Question",  '__index_level_0__'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/486 [00:00<?, ? examples/s]

Max target length: 187


In [61]:
def preprocess_function(sample,padding="max_length"):

    inputs = ["Answer the question : " + item for item in sample["Question"]]

  
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)


    labels = tokenizer(text_target=sample["Answer"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=['Question','Answer', '__index_level_0__'])
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=['Question','Answer', '__index_level_0__'])
print(f"Keys of tokenized dataset: {list(tokenized_train.features)}")

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [62]:
from transformers import DataCollatorForSeq2Seq


label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [63]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")


metric = evaluate.load("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]


    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bbsur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [65]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir=f"/model",
    predict_with_generate=True,
    fp16=False, 
    learning_rate=5e-5,
    num_train_epochs=50
)

In [66]:
from transformers import Seq2SeqTrainingArguments, TrainingArguments,Trainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [67]:
trainer.train()

C:\Users\bbsur\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.841900
1000,1.112300
1500,0.760000
2000,0.577200
2500,0.492500


TrainOutput(global_step=2750, training_loss=0.9117468233975498, metrics={'train_runtime': 35735.4989, 'train_samples_per_second': 0.611, 'train_steps_per_second': 0.077, 'total_flos': 1168902199296000.0, 'train_loss': 0.9117468233975498, 'epoch': 50.0})

In [68]:
trainer.evaluate()

{'eval_loss': 2.2701423168182373,
 'eval_rouge1': 43.8014,
 'eval_rouge2': 26.7838,
 'eval_rougeL': 39.7778,
 'eval_rougeLsum': 40.3461,
 'eval_gen_len': 17.775510204081634,
 'eval_runtime': 36.9774,
 'eval_samples_per_second': 1.325,
 'eval_steps_per_second': 0.189,
 'epoch': 50.0}

In [69]:
model.save_pretrained("faq_flant5_new_base")

In [70]:
tokenizer.save_pretrained("faq_flant5_new_base")

('faq_flant5_new_base\\tokenizer_config.json',
 'faq_flant5_new_base\\special_tokens_map.json',
 'faq_flant5_new_base\\spiece.model',
 'faq_flant5_new_base\\added_tokens.json',
 'faq_flant5_new_base\\tokenizer.json')

In [84]:
import  transformers
from transformers import AutoModel, AutoTokenizer

model = transformers.AutoModelForSeq2SeqLM.from_pretrained("faq_flant5_new")
tokenizer = AutoTokenizer.from_pretrained("faq_flant5_new")

In [85]:
input_question = "Answer the question : why should I prep for disaster?"
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [86]:
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

Preparing for disaster is essential to protect your family and property from damage during a disaster.


In [87]:
input_question = "Answer the question : Should i take battery in my emergency kit?"
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
#input_question_ids = tokenized_question['input_ids'].to(device)
#input_attention_mask = tokenized_question['attention_mask'].to(device)
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [88]:
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

Yes, you should include a battery-powered flashlight or battery-powered radio in your emergency kit.


In [89]:
input_question ="Answer the question : how to be stay informed in a disaster?"
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
#input_question_ids = tokenized_question['input_ids'].to(device)
#input_attention_mask = tokenized_question['attention_mask'].to(device)
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [90]:
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

Stay informed by following local and national news, following local authorities' instructions, and contacting local authorities for information.


In [95]:
input_question ="Answer the question : What 10 things should I keep in my emergency kit?"
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
#input_question_ids = tokenized_question['input_ids'].to(device)
#input_attention_mask = tokenized_question['attention_mask'].to(device)
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [96]:
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

Include a first-aid kit, flashlight, batteries, first-aid supplies, medications, and medications for pets in your emergency kit.


In [97]:
pip install flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\bbsur\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Define a route for generating responses to user queries
@app.route("/generate", methods=["POST"])
def generate():
    # Get the user query from the request body
    user_query = request.json["query"]
    user_query="Answer the question :"+user_query
    tokenized_question = tokenizer(user_query, truncation=True, padding=True, return_tensors='pt')
    output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Return the response to the user
    return jsonify({"response": response})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.103:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:48:10] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:48:13] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:48:44] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:49:34] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:50:15] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:51:46] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:54:02] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:55:12] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Oct/2023 12:56:06] "PO